## Scrape . Store . Display -- A trip to Mars
This project demonstrates the ability to scrape various websites for data, store the information found in a MongoDB and display the information found on one HTML page. 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser


** chromedriver needs to be added to the path **

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


# 1. Scrape

### NASA Mars News
Scrape the <a href = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest">NASA Mars News Site</a> and collect the latest News Title and Paragraph Text. Store titles and paragraphs in a dictionary.

In [2]:
# Open Browser
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome',**executable_path,headless = False)

In [4]:
# Visit NASA Mars News Site
nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(nasa_url)

In [5]:
# Scrape the title and paragraphs of the articles and store them in a dictionary
html = browser.html
soup = bs(html)
articles_dict = {"Title" : [], "Paragraph": []}
articles = soup.find_all("li",class_ = "slide")
for article in articles:
    articles_dict["Title"].append(article.find('div',class_ = "content_title").text)
    articles_dict["Paragraph"].append(article.find('div', class_ = "article_teaser_body").text)
#articles_df = pd.DataFrame(articles_dict) -- just in case to see what's in the dictionary -> df

### JPL Mars Space Images - Featured Image
Visit the <a href = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars">URL</a> for JPL Featured Space Image here. Use splinter to navigate the site and find the image url for the current Featured Mars Image. Assign the url string to a variable called featured_image_url.

In [8]:
jpl_img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_img_url)
html_jpl = browser.html
soup_jpl = bs(html_jpl)

In [9]:
start_url = 'https://www.jpl.nasa.gov'
featured_image_url = start_url + soup_jpl.find("a",class_="button")["data-fancybox-href"]
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18280_ip.jpg'

### Mars Weather
Visit the <a href = "https://twitter.com/marswxreport?lang=en">Mars Weather twitter account</a> here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

In [10]:
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)
soup_weather = bs(browser.html)

In [11]:
tweet = soup_weather.find('div',class_ = 'content').find('p')
unwanted = tweet.find('a')
unwanted.extract()
mars_weather = tweet.text

### Mars Facts
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc. Use Pandas to convert the data to a HTML table string.

In [9]:
facts_url = "https://space-facts.com/mars/"
facts_df = pd.read_html(facts_url)[0]
facts_df = facts_df.rename(columns = {0: "description", 1 : "value"})
facts_df = facts_df.set_index("description")
#table_facts_html = facts_df.to_html()
facts_df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres
Visit the <a href = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"> USGS Astrogeology site</a> to obtain high resolution images for each of Mar's hemispheres.You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [24]:
astrogeology_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(astrogeology_url)
soup_astro = bs(browser.html)

In [52]:
hemisphere_image_urls = []

In [54]:
hemisphere_hrefs = []
imgs = soup_astro.find_all("div",class_ = "item")
for div in imgs:
    hemisphere_hrefs.append(div.find('a')['href'])

In [56]:
start_url = "https://astrogeology.usgs.gov"
for href in hemisphere_hrefs:
    url = start_url + href
    browser.visit(url)
    soup_hemisphere = bs(browser.html)
    hemisphere = {"title" : soup_hemisphere.find('h2').text.strip(" Enhanced"),
    "img_url": soup_hemisphere.find('div',class_ = "downloads").find("a")["href"]}
    hemisphere_image_urls.append(hemisphere)

In [63]:
browser.quit()

# 2.Store & 3. Display

Please see mars_app.py

In [15]:
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client["marsDB"]
mars_docs = db.list_collection_names()

In [16]:
mars_docs

[]